In [1]:
from pystac_client import Client
import requests
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
forecast_api_key = 'e55dced6-70fb-44f1-a658-cdef62c74b6a'
climate_api_key = 'fa9056ec-2f41-4042-828b-91750e966966'
metobs_api_key = 'd40321ee-7de5-4417-a0bf-108bd34061ab'

In [3]:
base_url = 'https://dmigw.govcloud.dk/v2/climateData/collections/20kmGridValue/items'
parameter_ids =  ['mean_temp', 'mean_wind_speed', 'mean_wind_dir','max_wind_speed_10min']  # Add more parameters if needed
period = '2024-09-01T00:00:00Z/2024-10-01T00:00:00Z' # edit if needed

In [5]:
# Function to pull all data with pagination
def fetch_all_data(parameter_id, limit=300000):
    offset = 0  # Start with no offset
    all_data = []  # List to collect all records

    while True:
        # Make the API request with the offset
        params = {
            'parameterId': parameter_id,
            'api-key': climate_api_key,
            'datetime': period,
            'limit': limit,
            'offset': offset,
            'timeResolution': 'hour',
        }
        
        response = requests.get(base_url, params=params)
        response_json = response.json()

        # Extract the features from the response
        features = response_json.get('features', [])

        # If no more features are returned, we have fetched all the data
        if not features:
            break

        # Append the fetched features to the all_data list
        for feature in features:
            feature_data = {
                'cellId': feature['properties']['cellId'],
                'from': feature['properties']['from'],
                'to': feature['properties']['to'],
                parameter_id: feature['properties']['value'],  # Store the value under the parameter name
            }
            all_data.append(feature_data)

        # Increment the offset by the number of records returned in this batch
        offset += response_json.get('numberReturned', 0)

    return all_data

In [6]:
# Initialize an empty DataFrame to hold the final results
df_final = pd.DataFrame()

# Loop over each parameterId
for param_id in parameter_ids:
    # Fetch all data for the current parameterId using pagination
    data = fetch_all_data(param_id)

    # Convert the data to a DataFrame
    df_current = pd.DataFrame(data)

    # Merge the current DataFrame with the final one
    if df_final.empty:
        df_final = df_current  # First parameter: just initialize the final DataFrame
    else:
        df_final = pd.merge(df_final, df_current, on=['cellId', 'from', 'to'], how='outer')

In [12]:
df_final[(df_final['cellId']=='20km_604_64')]

,cellId,from,to,mean_temp,mean_wind_speed,mean_wind_dir,max_wind_speed_10min
0,20km_604_64,2024-09-01T00:00:00+00:00,2024-09-01T01:00:00+00:00,13.0,2.4,46.0,3.6
1,20km_604_64,2024-09-01T01:00:00+00:00,2024-09-01T02:00:00+00:00,12.8,2.1,47.0,2.8
2,20km_604_64,2024-09-01T02:00:00+00:00,2024-09-01T03:00:00+00:00,12.5,1.9,46.0,2.5
3,20km_604_64,2024-09-01T03:00:00+00:00,2024-09-01T04:00:00+00:00,12.2,1.9,48.0,2.5
4,20km_604_64,2024-09-01T04:00:00+00:00,2024-09-01T05:00:00+00:00,12.1,2.0,63.0,2.8
...,...,...,...,...,...,...,...
716,20km_604_64,2024-09-30T20:00:00+00:00,2024-09-30T21:00:00+00:00,12.0,9.3,116.0,10.4
717,20km_604_64,2024-09-30T21:00:00+00:00,2024-09-30T22:00:00+00:00,11.8,9.4,116.0,10.7
718,20km_604_64,2024-09-30T22:00:00+00:00,2024-09-30T23:00:00+00:00,11.7,9.3,115.0,10.7
719,20km_604_64,2024-09-30T23:00:00+00:00,2024-10-01T00:00:00+00:00,11.4,9.3,114.0,10.6


In [13]:
# finding mean wind speed is nan
df_final

,cellId,from,to,mean_temp,mean_wind_speed,mean_wind_dir,max_wind_speed_10min
0,20km_604_64,2024-09-01T00:00:00+00:00,2024-09-01T01:00:00+00:00,13.0,2.4,46.0,3.6
1,20km_604_64,2024-09-01T01:00:00+00:00,2024-09-01T02:00:00+00:00,12.8,2.1,47.0,2.8
2,20km_604_64,2024-09-01T02:00:00+00:00,2024-09-01T03:00:00+00:00,12.5,1.9,46.0,2.5
3,20km_604_64,2024-09-01T03:00:00+00:00,2024-09-01T04:00:00+00:00,12.2,1.9,48.0,2.5
4,20km_604_64,2024-09-01T04:00:00+00:00,2024-09-01T05:00:00+00:00,12.1,2.0,63.0,2.8
...,...,...,...,...,...,...,...
128333,20km_638_58,2024-09-30T20:00:00+00:00,2024-09-30T21:00:00+00:00,13.2,11.5,126.0,13.1
128334,20km_638_58,2024-09-30T21:00:00+00:00,2024-09-30T22:00:00+00:00,12.8,11.7,126.0,13.4
128335,20km_638_58,2024-09-30T22:00:00+00:00,2024-09-30T23:00:00+00:00,12.6,11.0,123.0,12.9
128336,20km_638_58,2024-09-30T23:00:00+00:00,2024-10-01T00:00:00+00:00,12.2,11.1,115.0,13.2
